In [4]:
import pickle
import numpy as np
import quimb.tensor as qtn
import quimb as qu
import scipy.linalg as la
from mpo_utils import mpo_to_mps, mps_to_mpo
from exact_construct import create_hamiltonian
from scipy.linalg import expm

In [5]:

L=6
J=1
h=1.5


In [6]:
split_opts={'method':'svd','cutoff':1e-9,'cutoff_mode':'rel','max_bond':100}
h2=-J*(qu.pauli('Z') & qu.pauli('I')) & (qu.pauli('Z') & qu.pauli('I'))
h1=h*qu.pauli('X') & qu.pauli('I')
local_ham = qtn.LocalHam1D(L=L, H2=h2, H1=h1,cyclic=False)
def gibbs_mpo(local_ham,tebd_tol,split_opts):
    tebd_tol=1e-10
    rho0=qtn.MPS_product_state([np.eye(2)]*L)
    tebd=qtn.TEBD(rho0,local_ham,imag=True,progbar=False,split_opts=split_opts)
    tebd.update_to(1,tol=tebd_tol)
    rho=mps_to_mpo(tebd.pt)
    rho=rho*(1/rho.trace())
    return rho
rho=gibbs_mpo(local_ham,1e-10,split_opts)

KeyboardInterrupt: 

In [4]:

x=np.array([[0,1],[1,0]])
z=np.array([[1,0],[0,-1]])
i=np.eye(2)
kraus_ops=[x,z]

h2 = -J*np.kron(np.kron(z,i),np.kron(z,i))
h1=h*np.kron(x,i)
local_ham = qtn.LocalHam1D(L=L, H2=-1j*h2, H1=-1j*h1,cyclic=False)

local_hamiltonians = dict(qtn.LocalHam1D(L=L, H2=-J*np.kron(z,z), H1=h*x,cyclic=False).items())



In [5]:
mps0 = mpo_to_mps(qtn.MPO_identity(L,sites=None, phys_dim=2, cyclic=False))
mps0


MatrixProductState(tensors=6, indices=11, L=6, max_bond=1)

In [6]:
tebd=qtn.tensor_1d_tebd.TEBD(mps0,local_ham,imag=False,tol=1e-7)

In [7]:
tebd.update_to(1)

t=0.01109, max-bond=2:   0%|          | 0/100 [00:00<?, ?%/s]

t=1, max-bond=7: 100%|##########| 100/100 [00:00<00:00, 145.95%/s]    


In [8]:
res=mps_to_mpo(tebd.pt)
rho_mpo=res/res.trace(res.lower_inds,res.upper_inds)


In [9]:
h=create_hamiltonian(local_hamiltonians)
exp=expm(-h)
rho=exp/np.trace(exp)

In [10]:
np.linalg.norm(rho-rho_mpo.to_dense(),ord=2)/np.linalg.norm(rho,ord=2)

np.float64(0.0006514689639042191)